Установить git (git-scm.com)

- pip install pymorphy2
- pip install git+https://github.com/nlpub/pymystem3
- pip install spacy_udpipe


In [28]:
# импорт всех необходимых модулей

import json
import pandas as pd
import re
import pymorphy2
import pymystem3
from corpy.udpipe import Model
from razdel import tokenize

In [30]:
# загрузим датасет

data = []
with open('google.json', encoding='utf8') as file:  # меняйте название файла на свое
    for line in file:
        data.append(json.loads(line))
data = pd.DataFrame(data)

In [6]:
data.head()

,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,...,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,1446016335457234944,1446016335457234944,2021-10-07 10:35:34 MSK,2021-10-07,10:35:34,+0300,53002114,googlerussia,Google Россия,,...,,,,,,[],,,,
1,1445395751274754061,1445395751274754061,2021-10-05 17:29:35 MSK,2021-10-05,17:29:35,+0300,53002114,googlerussia,Google Россия,,...,,,,,,[],,,,
2,1443587660560297985,1443183401192804362,2021-09-30 17:44:53 MSK,2021-09-30,17:44:53,+0300,53002114,googlerussia,Google Россия,,...,,,,,,[],,,,
3,1443183401192804362,1443183401192804362,2021-09-29 14:58:30 MSK,2021-09-29,14:58:30,+0300,53002114,googlerussia,Google Россия,,...,,,,,,[],,,,
4,1442487926399463426,1442487926399463426,2021-09-27 16:54:56 MSK,2021-09-27,16:54:56,+0300,53002114,googlerussia,Google Россия,,...,,,,,,[],,,,


In [3]:
data.columns

Index(['id', 'conversation_id', 'created_at', 'date', 'time', 'timezone',
       'user_id', 'username', 'name', 'place', 'tweet', 'language', 'mentions',
       'urls', 'photos', 'replies_count', 'retweets_count', 'likes_count',
       'hashtags', 'cashtags', 'link', 'retweet', 'quote_url', 'video',
       'thumbnail', 'near', 'geo', 'source', 'user_rt_id', 'user_rt',
       'retweet_id', 'reply_to', 'retweet_date', 'translate', 'trans_src',
       'trans_dest'],
      dtype='object')

In [6]:
data = data[data.language == 'ru']  # отфильтруем только русский

In [7]:
data.tweet = data.tweet.apply(lambda tweet: re.sub(r'https://.+|@\S+|#\S+', '', tweet))
# почистим твиты от адресов, обращений к людям (@google) и хештегов

In [8]:
pd.set_option('display.max_colwidth', None)  # эта штука нужна, чтобы отображались твиты целиком
data[['tweet']].head()

,tweet
0,"Кто такой «бөтүүк» и почему «кот» — не всегда животное? Пройди тест и проверь, насколько хорошо ты понимаешь языки разных народов России!"
1,"Сегодня — праздник тех, кто посвятил свою жизнь тому, чтобы делиться знаниями. Спасибо, что несмотря на непростые два года и непривычный онлайн-формат, вы продолжаете пробуждать любопытство и терпеливо отвечать на бесконечные «почему» и «как» своих учеников!"
2,"Для тех, кто пропустил онлайн-конференцию Search On, выпустили материал о всех обновлениях Google Поиск!"
3,"Узнай, как ИИ помогает нам сделать Google Поиск еще полезнее и удобнее. Сегодня, в 20:00, на онлайн-конференции Search On мы поделимся нашими новостями. Не пропусти!"
4,С днём рождения нас! Сегодня нам исполнилось 23 года 🎉


Используем правиловый токенизатор pymorphy2

In [9]:
tokens = []
for elem in data.tweet:
    tokens.extend([t.text for t in tokenize(elem)])
tokens[:10]

['Кто', 'такой', '«', 'бөтүүк', '»', 'и', 'почему', '«', 'кот', '»']

In [10]:
res = []
morph = pymorphy2.MorphAnalyzer()
for token in tokens[:1000]:
    res.append(morph.parse(token)[0])
res[:10]

[Parse(word='кто', tag=OpencorporaTag('NPRO,masc sing,nomn'), normal_form='кто', score=1.0, methods_stack=((DictionaryAnalyzer(), 'кто', 1817, 0),)),
 Parse(word='такой', tag=OpencorporaTag('ADJF,Apro femn,sing,gent'), normal_form='такой', score=0.51004, methods_stack=((DictionaryAnalyzer(), 'такой', 1754, 8),)),
 Parse(word='«', tag=OpencorporaTag('PNCT'), normal_form='«', score=1.0, methods_stack=((PunctuationAnalyzer(score=0.9), '«'),)),
 Parse(word='бөтүүк', tag=OpencorporaTag('NOUN,anim,masc sing,nomn'), normal_form='бөтүүк', score=0.928456332581373, methods_stack=((FakeDictionary(), 'бөтүүк', 2, 0), (KnownSuffixAnalyzer(min_word_length=4, score_multiplier=0.5), 'к'))),
 Parse(word='»', tag=OpencorporaTag('PNCT'), normal_form='»', score=1.0, methods_stack=((PunctuationAnalyzer(score=0.9), '»'),)),
 Parse(word='и', tag=OpencorporaTag('CONJ'), normal_form='и', score=0.998263, methods_stack=((DictionaryAnalyzer(), 'и', 20, 0),)),
 Parse(word='почему', tag=OpencorporaTag('ADVB,Ques'),

In [13]:
# чтобы собрать разборы и записать их в табличку в экселе:

pymorphyres = pd.DataFrame([{'word': t.word, 
                'lemma': t.normal_form, 
                'pos': t.tag.POS, 
                'feats': t.tag} for t in res])
pymorphyres.head()

,word,lemma,pos,feats
0,кто,кто,NPRO,"NPRO,masc sing,nomn"
1,такой,такой,ADJF,"ADJF,Apro femn,sing,gent"
2,«,«,None,PNCT
3,бөтүүк,бөтүүк,NOUN,"NOUN,anim,masc sing,nomn"
4,»,»,None,PNCT


In [14]:
pymorphyres.to_excel('ru_pymorphy.xlsx')

А теперь майстем

In [19]:
mystem = pymystem3.Mystem(entire_input=False, glue_grammar_info=False)

resM = []
for elem in data.loc[:1000, 'tweet']:
    resM.extend(mystem.analyze(elem))
    
resM[:10]

[{'analysis': [{'lex': 'кто', 'gr': 'SPRO,ед,муж,од=им'}], 'text': 'Кто'},
 {'analysis': [{'lex': 'такой', 'gr': 'APRO=пр,ед,жен'},
   {'lex': 'такой', 'gr': 'APRO=дат,ед,жен'},
   {'lex': 'такой', 'gr': 'APRO=род,ед,жен'},
   {'lex': 'такой', 'gr': 'APRO=твор,ед,жен'},
   {'lex': 'такой', 'gr': 'APRO=вин,ед,муж,неод'},
   {'lex': 'такой', 'gr': 'APRO=им,ед,муж'}],
  'text': 'такой'},
 {'analysis': [], 'text': 'бөтүүк'},
 {'analysis': [{'lex': 'и', 'gr': 'CONJ='}], 'text': 'и'},
 {'analysis': [{'lex': 'почему', 'gr': 'ADVPRO='}], 'text': 'почему'},
 {'analysis': [{'lex': 'кот', 'gr': 'S,муж,од=им,ед'}], 'text': 'кот'},
 {'analysis': [{'lex': 'не', 'gr': 'PART='}], 'text': 'не'},
 {'analysis': [{'lex': 'всегда', 'gr': 'ADVPRO='}], 'text': 'всегда'},
 {'analysis': [{'lex': 'животное', 'gr': 'S,сред,од=вин,ед'},
   {'lex': 'животное', 'gr': 'S,сред,од=им,ед'}],
  'text': 'животное'},
 {'analysis': [{'lex': 'проходить', 'gr': 'V,нп=ед,пов,2-л,сов'}],
  'text': 'Пройди'}]

In [20]:
mystemresults = []
for token in resM:
    if token.get('analysis'):
        mystemresults.append({'word': token['text'],
                             'lemma': token['analysis'][0]['lex'],
                             'pos': mystem.get_pos(token),
                             'morpho': token['analysis'][0]['gr']})
    else:
        mystemresults.append({'word': token['text'],
                             'lemma': token['text'],
                             'pos': None,
                             'morpho': None})
mystemresults = pd.DataFrame(mystemresults)
mystemresults.head()

,word,lemma,pos,morpho
0,Кто,кто,SPRO,"SPRO,ед,муж,од=им"
1,такой,такой,APRO,"APRO=пр,ед,жен"
2,бөтүүк,бөтүүк,None,None
3,и,и,CONJ,CONJ=
4,почему,почему,ADVPRO,ADVPRO=


In [21]:
mystemresults.to_excel('ru_mystem.xlsx')

А теперь udpipe - парсер на нейронной сети

In [23]:
m = Model('russian-taiga-ud-2.5-191206.udpipe')  # загрузим модель udpipe для русского языка

In [24]:
sents = list(m.process(data.loc[0, 'tweet'])) # это тестовый прогон на единственном твите

In [25]:
for sent in sents:
    for word in sent.words:
        print(word.form, word.lemma, word.feats)

<root> <root> <root>
Кто кто Animacy=Anim|Case=Nom|Gender=Masc|Number=Sing
такой такой Case=Nom|Gender=Masc|Number=Sing
« « 
бөтүүк бөтүүк Animacy=Anim|Case=Nom|Gender=Masc|Number=Sing
» » 
и и 
почему почему Degree=Pos
« « 
кот кот 
» » 
— — 
не не Polarity=Neg
всегда всегда Degree=Pos
животное животный Case=Nom|Degree=Pos|Gender=Neut|Number=Sing
? ? 
<root> <root> <root>
Пройди Пройди Aspect=Imp|Mood=Imp|Number=Sing|Person=2|VerbForm=Fin|Voice=Act
тест тест Animacy=Inan|Case=Acc|Gender=Masc|Number=Sing
и и 
проверь проверь Aspect=Imp|VerbForm=Inf|Voice=Act
, , 
насколько насколько 
хорошо хорошо Degree=Pos
ты ты Case=Nom|Number=Sing|Person=2
понимаешь понимать Aspect=Imp|Mood=Ind|Number=Sing|Person=2|Tense=Pres|VerbForm=Fin|Voice=Act
языки язык Animacy=Inan|Case=Nom|Gender=Masc|Number=Plur
разных разный Case=Gen|Degree=Pos|Number=Plur
народов народ Animacy=Inan|Case=Gen|Gender=Masc|Number=Plur
России Россия Animacy=Inan|Case=Gen|Gender=Fem|Number=Sing
! ! 


In [26]:
# чтобы прогнать на тысяче твитов и записать:

udresults = []
for tweet in data.loc[:1000, 'tweet']:
    for sent in m.process(tweet):
        for word in sent.words:
            udresults.append({'word': word.form, 'lemma': word.lemma, 'pos': word.upostag, 'morph': word.feats})
            
udresults = pd.DataFrame(udresults)
udresults.head()

,word,lemma,pos,morph
0,<root>,<root>,<root>,<root>
1,Кто,кто,PRON,Animacy=Anim|Case=Nom|Gender=Masc|Number=Sing
2,такой,такой,DET,Case=Nom|Gender=Masc|Number=Sing
3,«,«,PUNCT,
4,бөтүүк,бөтүүк,NOUN,Animacy=Anim|Case=Nom|Gender=Masc|Number=Sing


In [27]:
udresults.to_excel('udrussian.xlsx')